<a href="https://colab.research.google.com/github/amrosnik/adventure_scrape/blob/master/INFINNERATOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

InFINNerator: an RNN for generating Finn the Human speech

Here's my first attempt at creating an RNN. I will be using a homemade dataset of Adventure Time transcripts of Finn dialogue to generate "new" Finn speech! 

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

Let's first look at the data. I've outlined elsewhere how I formatted the dataset, but for now let's unpack the pickled Finn data.

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import unicodedata
import string
import pickle
import random
import numpy as np
import cloudpickle as cp
from urllib.request import urlopen
import time
import math
import torch.optim as optim

## ripped from https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html 
torch.manual_seed(1)
all_letters = string.printable #string.ascii_letters + " .,;'!?-"
n_letters = len(all_letters)+1
url = 'https://github.com/amrosnik/adventure_scrape/raw/master/finn_just_dialogue.pkl'
finn_data = cp.load(urlopen(url)) 

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )
finn_text = [unicodeToAscii(line) for line in finn_data]
finn_dialogue = [line for line in finn_text if len(line) > 0 ]
random.shuffle(finn_dialogue)

# concatenate all lines into one big corpus:
finn_corpus = '\n'.join(str(elem) for elem in finn_dialogue)
finn_corpus = list(finn_corpus)
finn_corpus = np.asarray(finn_corpus)

## separate dataset into lines of equal number of chars 
line_size = 64
num_lines = round(len(finn_corpus)/line_size)
#print(num_lines)
randomized_finn = [(finn_corpus[i:i+line_size]) for i in range(0, len(finn_corpus), line_size)]
#print(randomized_finn[0])
#print(randomized_finn[400])

# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
# One-hot matrix of first to last letters (not including EOS) for input
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

# LongTensor of second letter to end (EOS) for target
def targetTensor(line):
    letter_indexes = [all_letters.find(line[li]) for li in range(1, len(line))]
    return torch.LongTensor(letter_indexes)

   # Random item from a list
def randomChoice(l):
      return l[random.randint(0, len(l) - 1)]

# Make category, input, and target tensors from a random category, line pair
def randomTrainingExample(minibatch_k=32):
    limits = len(finn_corpus)-(line_size+1)
    idxs = np.asarray(np.random.randint(0,limits,size=minibatch_k))
    inputs = np.array([finn_corpus[idx:idx+line_size] for idx in idxs])
    targets = np.array([finn_corpus[(idx+1):idx+1+line_size] for idx in idxs])
    inputs = inputs.T
    targets = targets.T
    #print(inputs.shape,targets.shape,len(inputs))
    input_line_tensor = torch.zeros(line_size, minibatch_k, n_letters)  ## size 64,32,101
    for line in range(len(inputs[0])):
       for li, letter in enumerate(inputs[line]):
          #print(i,line,letter)
          input_line_tensor[li][line][letterToIndex(letter)] = 1
    target_line_tensor = torch.zeros(line_size, minibatch_k,dtype=torch.long)
    for line in range(len(targets[0])):
       for li,letter in enumerate(targets[line]):
          target_line_tensor[li][line] = letterToIndex(letter)
    #target_line_tensor.long()# = torch.LongTensor(target_line_tensor)
    #line = randomChoice(randomized_finn)
    #input_line_tensor = lineToTensor(line)
    #target_line_tensor = targetTensor(line)
    #print(input_line_tensor, target_line_tensor)
    #print(input_line_tensor.size(), target_line_tensor.size()) 
    return input_line_tensor, target_line_tensor
  
test,test2 = randomTrainingExample()
#for i in range(10):
#    input_line_tensor, target_line_tensor = randomTrainingExample()
#    print(input_line_tensor.size(), target_line_tensor.size())  

In [0]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        #self.o2o = nn.Linear(hidden_size + output_size, output_size)
        #self.dropout = nn.Dropout(0.1)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        input_combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(input_combined)
        output = self.i2o(input_combined)
        #output_combined = torch.cat((hidden, output), 1)
        #output = self.o2o(output_combined)
        #output = self.dropout(output)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self,batch_size):
        return torch.zeros(batch_size, self.hidden_size)
            

In [89]:
criterion = nn.NLLLoss()

learning_rate = 0.001
rnn = RNN(n_letters, 128, n_letters)
optimizer = optim.Adam(rnn.parameters(), lr=learning_rate)

# or... 
#rnn = nn.LSTM(10, 20, 2) https://pytorch.org/docs/stable/nn.html#lstm

def train(input_line_tensor, target_line_tensor):
    #print('before ',target_line_tensor.size())
    #target_line_tensor.unsqueeze_(-1) #OG code: target_line_tensor.unsqueeze_(-1)
    #print(target_line_tensor.size())
    hidden = rnn.initHidden(input_line_tensor.size(1))

    #rnn.zero_grad()
    optimizer.zero_grad()

    #loss = 0
    loss = torch.empty(1)
    #print('before loop',loss)
    output = torch.empty((input_line_tensor.size(1),input_line_tensor.size(2)),requires_grad=True)
    #print(input_line_tensor.size(0))

    #print(target_line_tensor)
    for i in range(input_line_tensor.size(0)):
        #print(i,input_line_tensor[i].size(),input_line_tensor.size(0))
        output, hidden = rnn(input_line_tensor[i], hidden)
        #print('output who dis',output.size())
        #print(i,input_line_tensor[i],
        #print(target_line_tensor[i])
        #print(target_line_tensor[i].size())
        l = criterion(output, target_line_tensor[i])
        #print('l',l)
        loss += l
        #print(l,loss)

    #print('after loop',loss)
    loss.requires_grad_()
    #print('after loop II',loss)
    loss.backward()

    optimizer.step()
    #for p in rnn.parameters():
    #    p.data.add_(-learning_rate, p.grad.data)

    #print(input_line_tensor.size(0), loss.item() / input_line_tensor.size(0))
    return output, loss.item() / input_line_tensor.size(0)
    
def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)
  
n_iters = 50000
print_every = 500
plot_every = 50
all_losses = []
total_loss = 0 # Reset every plot_every iters

start = time.time()

for iter in range(1, n_iters + 1):
    output, loss = train(*randomTrainingExample())
    total_loss += loss
    #print(loss)
    
    #print('out',loss)
    if iter % print_every == 0:
        print('%s (%d %d%%) %.4f' % (timeSince(start), iter, iter / n_iters * 100, loss))

    if iter % plot_every == 0:
        all_losses.append(total_loss / plot_every)
        total_loss = 0


0m 32s (500 1%) 1.5119
1m 6s (1000 2%) 1.5670
1m 38s (1500 3%) 1.3594
2m 15s (2000 4%) 1.2988
2m 59s (2500 5%) 1.3364
3m 42s (3000 6%) 1.2382
4m 24s (3500 7%) 1.2341
5m 5s (4000 8%) 1.2251
5m 42s (4500 9%) 1.2044
6m 13s (5000 10%) 2.3029
6m 39s (5500 11%) 1.3979
7m 5s (6000 12%) 1.2686
7m 30s (6500 13%) 1.2536
7m 57s (7000 14%) 1.2311
8m 23s (7500 15%) 1.2381
8m 50s (8000 16%) 1.2290
9m 24s (8500 17%) 1.2105
9m 58s (9000 18%) 1.2109
10m 31s (9500 19%) 1.1780
11m 9s (10000 20%) 1.2359
11m 55s (10500 21%) 1.2115
12m 38s (11000 22%) 1.2211
13m 15s (11500 23%) 1.2387
13m 42s (12000 24%) 1.2129


KeyboardInterrupt: ignored

In [96]:
max_length = 100
print(output)
exp_output = torch.exp(output)
for i in range(len(exp_output)):
   print(torch.sum(exp_output[i]))
# Sample from a category and starting letter
def sample(start_letter='A'):
    with torch.no_grad():  # no need to track history in sampling
        input = lineToTensor(start_letter)
        hidden = rnn.initHidden(1)

        output_name = start_letter

        for i in range(max_length):
            output, hidden = rnn(input[0], hidden)
            exp_output = torch.exp(output)
            #topv, topi = output.topk(1)
            #topi = topi[0][0]
            #if topi == n_letters - 1:
            #    break
            #else:
            #    letter = all_letters[topi]
            #    output_name += letter
            input = lineToTensor(letter)

        return output_name

# Get multiple samples from one category and multiple starting letters
def samples(start_letters='ABC'):
    for start_letter in start_letters:
        print(sample(start_letter))
  
start_index = vectorizer.surname_vocab.start_index
batch_size = 32
hidden_size = 128
initial_h = Variable(torch.ones(batch_size, hidden_size))
initial_x_index = Variable(torch.ones(batch_size).long()) * start_index










samples('sds')

tensor([[  0.0000, -31.3449, -43.5888,  ..., -57.3696, -48.6913, -59.3628],
        [  0.0000, -35.2484, -49.1968,  ..., -64.7299, -54.8414, -66.9797],
        [  0.0000, -35.4786, -49.5302,  ..., -65.1741, -55.2216, -67.4418],
        ...,
        [  0.0000, -33.7315, -47.0195,  ..., -61.8688, -52.4521, -64.0198],
        [  0.0000, -39.1065, -54.7428,  ..., -72.0140, -60.9373, -74.5206],
        [  0.0000, -31.9100, -44.4024,  ..., -58.4362, -49.5847, -60.4675]],
       grad_fn=<LogSoftmaxBackward>)
tensor(1., grad_fn=<SumBackward0>)
tensor(1., grad_fn=<SumBackward0>)
tensor(1., grad_fn=<SumBackward0>)
tensor(1., grad_fn=<SumBackward0>)
tensor(1., grad_fn=<SumBackward0>)
tensor(1., grad_fn=<SumBackward0>)
tensor(1., grad_fn=<SumBackward0>)
tensor(1., grad_fn=<SumBackward0>)
tensor(1., grad_fn=<SumBackward0>)
tensor(1., grad_fn=<SumBackward0>)
tensor(1., grad_fn=<SumBackward0>)
tensor(1., grad_fn=<SumBackward0>)
tensor(1., grad_fn=<SumBackward0>)
tensor(1., grad_fn=<SumBackward0>)
ten

NameError: ignored